In [115]:
# importing python libraries for analysis
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import openpyxl

In [116]:
# Show all columns(instead of cascading columns in the middle)
pd.set_option("display.max_columns",None)
# Don't show numbers in scientific notation
pd.set_option("display.float_format","{:.2f}".format)

In [117]:
# reading the csv and xlsx file
df_customer = pd.read_csv("QVI_purchase_behaviour.csv")
df_transactions =pd.read_excel("QVI_transaction_data.xlsx")


In [118]:
# Converting excel data with pandas
df_transactions['DATE'] = pd.to_datetime(df_transactions['DATE'], unit='d', origin='1899-12-30')

In [119]:
#Displaying the data
df_customer.head(5)

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream


In [120]:
df_transactions.head(5)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.00
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.30
2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.90
3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.00
4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.80


In [121]:
#list of top selling products.
sales = df_transactions.groupby('PROD_NAME')['TOT_SALES'].sum() 

In [122]:
sales

PROD_NAME
Burger Rings 220g                        6831.00
CCs Nacho Cheese    175g                 5961.90
CCs Original 175g                        6048.00
CCs Tasty Cheese    175g                 6069.00
Cheetos Chs & Bacon Balls 190g           9243.30
                                           ...  
WW Sour Cream &OnionStacked Chips 160g   5323.80
WW Supreme Cheese   Corn Chips 200g      5390.30
Woolworths Cheese   Rings 190g           5169.60
Woolworths Medium   Salsa 300g           4050.00
Woolworths Mild     Salsa 300g           4234.50
Name: TOT_SALES, Length: 114, dtype: float64

In [123]:
#Printing column names, types, and non-null values
df_customer.info(verbose = True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72637 entries, 0 to 72636
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LYLTY_CARD_NBR    72637 non-null  int64 
 1   LIFESTAGE         72637 non-null  object
 2   PREMIUM_CUSTOMER  72637 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [124]:
#Printing column names, types, and non-null values
df_transactions.info(verbose = True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   DATE            264836 non-null  datetime64[ns]
 1   STORE_NBR       264836 non-null  int64         
 2   LYLTY_CARD_NBR  264836 non-null  int64         
 3   TXN_ID          264836 non-null  int64         
 4   PROD_NBR        264836 non-null  int64         
 5   PROD_NAME       264836 non-null  object        
 6   PROD_QTY        264836 non-null  int64         
 7   TOT_SALES       264836 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(5), object(1)
memory usage: 16.2+ MB


In [125]:
df_customer.shape

(72637, 3)

In [126]:
df_transactions.shape

(264836, 8)

In [127]:
#looking for duplicates in customer data
df_customer.duplicated().sum()

0

In [128]:
#looking for duplicates in the dataframe for transactions
df_transactions.duplicated().sum()

1

In [129]:
# identifying the duplicates
df_transactions.loc[df_transactions.duplicated(keep = 'first'),:]

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
124845,2018-10-01,107,107024,108462,45,Smiths Thinly Cut Roast Chicken 175g,2,6.00


In [130]:
#dropping duplicates
df_transactions.drop_duplicates(keep = 'first').shape

(264835, 8)

In [131]:
# Changing data type
df_transactions = df_transactions.astype({"PROD_NAME" : "category"})

In [132]:
df_transactions['PROD_NAME'].dtype

CategoricalDtype(categories=['Burger Rings 220g', 'CCs Nacho Cheese    175g',
                  'CCs Original 175g', 'CCs Tasty Cheese    175g',
                  'Cheetos Chs & Bacon Balls 190g', 'Cheetos Puffs 165g',
                  'Cheezels Cheese 330g', 'Cheezels Cheese Box 125g',
                  'Cobs Popd Sea Salt  Chips 110g',
                  'Cobs Popd Sour Crm  &Chives Chips 110g',
                  ...
                  'WW Crinkle Cut      Chicken 175g',
                  'WW Crinkle Cut      Original 175g',
                  'WW D/Style Chip     Sea Salt 200g',
                  'WW Original Corn    Chips 200g',
                  'WW Original Stacked Chips 160g',
                  'WW Sour Cream &OnionStacked Chips 160g',
                  'WW Supreme Cheese   Corn Chips 200g',
                  'Woolworths Cheese   Rings 190g',
                  'Woolworths Medium   Salsa 300g',
                  'Woolworths Mild     Salsa 300g'],
, ordered=False)

In [133]:
# Separating the weight from the product
df_transactions['WEIGHT']= df_transactions['PROD_NAME'].str[-4:] 

In [134]:
#Changing the order of the dataframe so that the weight is next to the product
df_transactions = df_transactions.iloc[:,[0,1,2,3,4,5,8,6,7]]

In [135]:
#Removing the weight
df_transactions['PROD_NAME'] = df_transactions['PROD_NAME'].str[:-4]

In [136]:
df_transactions

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,WEIGHT,PROD_QTY,TOT_SALES
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt,175g,2,6.00
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese,175g,3,6.30
2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken,170g,2,2.90
3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion,175g,5,15.00
4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili,150g,3,13.80
...,...,...,...,...,...,...,...,...,...
264831,2019-03-09,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream,175g,2,10.80
264832,2018-08-13,272,272358,270154,74,Tostitos Splash Of Lime,175g,1,4.40
264833,2018-11-06,272,272379,270187,51,Doritos Mexicana,170g,2,8.80
264834,2018-12-27,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno,150g,2,7.80


In [152]:
df_transactions.describe()

,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,264836.00,264836.00,264836.00,264836.00,264836.00,264836.00
mean,135.08,135549.48,135158.31,56.58,1.91,7.30
std,76.78,80579.98,78133.03,32.83,0.64,3.08
min,1.00,1000.00,1.00,1.00,1.00,1.50
25%,70.00,70021.00,67601.50,28.00,2.00,5.40
50%,130.00,130357.50,135137.50,56.00,2.00,7.40
75%,203.00,203094.25,202701.25,85.00,2.00,9.20
max,272.00,2373711.00,2415841.00,114.00,200.00,650.00


In [137]:
df_transactions["PROD_NAME"]

0           Natural Chip        Compny SeaSalt
1                         CCs Nacho Cheese    
2           Smiths Crinkle Cut  Chips Chicken 
3           Smiths Chip Thinly  S/Cream&Onion 
4         Kettle Tortilla ChpsHny&Jlpno Chili 
                          ...                 
264831     Kettle Sweet Chilli And Sour Cream 
264832               Tostitos Splash Of  Lime 
264833                    Doritos Mexicana    
264834     Doritos Corn Chip Mexican Jalapeno 
264835               Tostitos Splash Of  Lime 
Name: PROD_NAME, Length: 264836, dtype: object

In [138]:
# Merge the datasets
df_custrans = pd.merge(df_customer,df_transactions,on = "LYLTY_CARD_NBR", how = "outer")

In [156]:
df_custrans.head()

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,WEIGHT,PROD_QTY,TOT_SALES
0,1000,YOUNG SINGLES/COUPLES,Premium,2018-10-17,1,1,5,Natural Chip Compny SeaSalt,175g,2,6.00
1,1002,YOUNG SINGLES/COUPLES,Mainstream,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli,150g,1,2.70
2,1003,YOUNG FAMILIES,Budget,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives,210G,1,3.60
3,1003,YOUNG FAMILIES,Budget,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn,175g,1,3.00
4,1004,OLDER SINGLES/COUPLES,Mainstream,2018-11-02,1,5,96,WW Original Stacked Chips,160g,1,1.90


In [140]:
# most bought
#df_popular= (df_custrans.groupby(['PROD_NAME','WEIGHT','PROD_QTY'])['PROD_QTY'].agg(['count']).sort_values(by='count', ascending=False).reset_index().drop_duplicates('PROD_NAME', keep='first'))

In [162]:
#Calculating top sellers
df_topsellers = df_custrans.groupby(by=['PROD_NAME','WEIGHT'], as_index=False)['TOT_SALES'].sum().sort_values(by = 'TOT_SALES',ascending = False)

In [163]:
df_topsellers

,PROD_NAME,WEIGHT,TOT_SALES
11,Dorito Corn Chp Supreme,380g,40352.00
86,Smiths Crnkle Chip Orgnl Big Bag,380g,36367.60
77,Smiths Crinkle Chips Salt & Vinegar,330g,34804.20
33,Kettle Mozzarella Basil & Pesto,175g,34457.40
76,Smiths Crinkle Original,330g,34302.60
...,...,...,...
104,WW Crinkle Cut Chicken,175g,4702.20
90,Sunbites Whlegrn Crisps Frch/Onin,90g,4600.20
105,WW Crinkle Cut Original,175g,4532.20
113,Woolworths Mild Salsa,300g,4234.50
